In [1]:
'''
Enter the Title of the movie
Then click on "CELL->Run All" 
'''
title ="Prestige"

In [2]:
import re
import sys
from bs4 import BeautifulSoup
import requests
import time
import tweepy
from textblob import TextBlob
import pandas as pd
import model as m
from tweepy import OAuthHandler
import pandas as pd
import json
import pickle
import csv
%matplotlib inline

In [3]:
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = '34Z3k2QwDjvWb8HjgxeOT8h47'
        consumer_secret = '9dqRK0OVG5LAOTImEYMJC6N9fNEjolk16trsdiUaQpslr1Uv6E'
        access_token = '790131625287823360-1JOhf6LdjADGdojW1NRv6ywaHC9ifiL'
        access_token_secret = '16fkYIubpqihIMoSleEgf8SVB2QCgVgYK90XkreKNlk33'
 
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
 
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
 
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
 
    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
 
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
 
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
 
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
 
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
 
            # return parsed tweets
            return tweets
 
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))

def biggest(a, y, z):
    Max = a
    if y > Max:
        Max = y
    if z > Max:
        Max = z
        if y > z:
            Max = y
    if Max == a: return 1
    if Max == y: return 0
    if Max == z: return -1
                                                                                                                                                                                                                                           
    

# creating object of TwitterClient Class
api = TwitterClient()
# calling function to get tweets
tweets = api.get_tweets(query = title, count = 1000000)
 
# picking positive tweets from tweets
ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
# percentage of positive tweets
print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
# picking negative tweets from tweets
ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
# percentage of negative tweets
print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
# percentage of neutral tweets
print("Neutral tweets percentage: {} % \
    ".format(100* (len(tweets) - len(ntweets) - len(ptweets))/len(tweets)))
    
nutweets = len(tweets) - len(ntweets) - len(ptweets)
    
Sentiments = biggest(len(ptweets),nutweets,len(ntweets))
print(Sentiments)


Positive tweets percentage: 37.634408602150536 %
Negative tweets percentage: 10.75268817204301 %
Neutral tweets percentage: 51.61290322580645 %     
0


In [4]:
data= []
data.append(1)
data.append(title)

def popularity_fetch(url):
    try:
        res = requests.get(url)
        json_data = json.loads(res.text)
        return(json_data["results"][0]["popularity"])
    except requests.exceptions.RequestException as e:
        print(e)

def actor_popularity_fetch(url):
    try:
        res = requests.get(url)
        json_dat = json.loads(res.text)
        return(json_dat["results"][0]["popularity"])
    except requests.exceptions.RequestException as e:
        print(e)

movie = (re.sub(r'^[Tt]he',"", re.sub('[^\w\d-]',"", re.sub(r':.*',"",title))).lower() + ".htm")
url = 'http://www.boxofficemojo.com/movies/?id=' + str(movie)
print(url)
soupObjects={}  
try:
    resp = requests.get(url)
    soupObjects = BeautifulSoup(resp.text, 'lxml')
except requests.exceptions.RequestException as e:
    print(e)


pop_url = 'http://api.tmdb.org/3/search/person?api_key=d7721dd4346490aa61af275d888d2585&query='

# Distributor from bomojo
distributo = soupObjects.find(text=re.compile('Distributor:'))
if distributo:
    distributo = str(distributo.findNextSibling().text)
    data.append(distributo)

# genre from bomojo
genr = soupObjects.find(text=re.compile('Genre:'))
if genr:
    genr = str(genr.findNextSibling().text)
    data.append(genr)

# Actor from bomojo
act = soupObjects.find(text=re.compile('Actors:'))
if act:
    actor = act.next_element()[1].text
    data.append(actor)
    a_op_url= pop_url + act.next_element()[1].text
    actor_popularit = actor_popularity_fetch(a_op_url)
    data.append(actor_popularit)


# director from bomojo
direc = soupObjects.find(text=re.compile('Director:'))
if direc:
    direct = direc.next_element()[0].text
    data.append(direct)
    d_op_url= pop_url + direc.next_element()[0].text
    director_popularity = popularity_fetch(d_op_url)
    data.append(director_popularity)

data.append(Sentiments)


# Number of theatres from bomojo
num_of_theatres = soupObjects.find(text=re.compile('Widest'))
if num_of_theatres:
    num_of_theatres = str(num_of_theatres.next_element.next_element.text)
    num_of_theatres = int(num_of_theatres[:-8].replace(',',''))
    data.append(num_of_theatres)

# runtime
runtime = soupObjects.find(text=re.compile('Runtime:'))
if runtime:
    run_time = str(runtime.findNextSibling().text)
    run_time = run_time.replace('N/A','0')
    runtime_mins = float(run_time.split(' ')[0])*60 + float(run_time.split(' ')[2])
    data.append(runtime_mins)

# Release year from bomojo
release_year = soupObjects.find(text=re.compile('Release Date:'))
if release_year:
    release_year = str(release_year.findNextSibling().text)[-4:]
    data.append(int(release_year))
    
    
# production budget
pb = soupObjects.find(text=re.compile('Production Budget:'))    
if pb:
    prod_budget = str(pb.findNextSibling().text)
    prod_budget = prod_budget.replace('million','000000')
    prod_budget = prod_budget.replace('N/A|NA','0')
    prod_budget = prod_budget.replace(' ','')
    prod_budget = prod_budget.replace('$','')
    prod_budget = float(prod_budget)
    data.append(8.83E-06)
    cpi = 200

data.append(cpi)
data.append(prod_budget)                                                                                                                                                              
print(data)
data.append(int(prod_budget)^int(director_popularity) + cpi*num_of_theatres)

http://www.boxofficemojo.com/movies/?id=prestige.htm
[1, 'Prestige', 'Buena Vista', 'Fantasy Drama', 'Andy Serkis', 6.556776, 'Christopher Nolan', 7.479288, 0, 2305, 135.0, 2006, 8.83e-06, 200, 40000000.0]


In [5]:
with open('Data.csv','a') as f:
    writer=csv.writer(f)                                                                                                                                                                                                 
    writer.writerow([])
    writer.writerow(data)
    f.close()
dataset = pd.read_csv('Data.csv')    
df = pd.DataFrame(dataset)
df

,Id,title,distributor,Genre,leadActor,Actor popularity,director,Director popularity,Sentiment,num_theaters,runtime_mins,releaseYear,CULT_INDEX,CPI,prod_budget_ADJ,rev_totalGross_ADJ
0,3468,Whipped,Destination,Comedy,Amanda Peet,7.599365,Peter Cohen,1.120000,-1,1581,82.0,2000,5.530000e-10,172.200,4.146341e+06,1.090221e+07
1,3384,Little Nicky,New Line,Comedy,Adam Sandler,7.883535,Steven Brill,1.841047,-1,2910,84.0,2000,5.260000e-11,172.200,1.174797e+08,1.034267e+08
2,3381,The Ladies Man,Paramount,Comedy,Tim Meadows,1.939280,Reginald Hudlin,1.618813,-1,2022,84.0,2000,2.290000e-10,172.200,3.317073e+07,3.568550e+07
3,3448,The Tao of Steve,Sony Classics,Comedy,Donal Logue,3.748402,Jenniphr Goodman,1.400000,1,9,87.0,2000,5.730000e-05,172.200,1.415128e+06,1.137976e+07
4,3455,Turn It Up,New Line,Music Drama,Pras,1.156000,Robert Adetuyi,1.178253,1,661,87.0,2000,5.670000e-09,172.200,1.796748e+07,3.270542e+06
5,3337,Cecil B. Demented,Artisan,Comedy,Melanie Griffith,4.050260,John Waters,2.655440,1,9,88.0,2000,8.830000e-06,172.200,4.186671e+05,3.366715e+06
6,3344,The Crew,Buena Vista,Crime Comedy,Richard Dreyfuss,2.677116,Michael Dinner,1.063944,1,1510,88.0,2000,5.250000e-10,172.200,5.252033e+07,3.413001e+07
7,3347,Down to You,Miramax,Romantic Comedy,Freddie Prinze,2.818751,Kris Isacsson,1.008891,1,1971,89.0,2000,1.760000e-10,172.200,4.837398e+07,5.259552e+07
8,3465,Whatever It Takes,Sony / Columbia,Romantic Comedy,Shane West,5.635719,David Raynr,1.004854,1,2272,94.0,2000,2.290000e-10,172.200,4.422764e+07,2.292010e+07
9,3369,Hanging Up,Sony / Columbia,Comedy / Drama,Meg Ryan,4.863958,Diane Keaton,6.070353,1,2618,95.0,2000,7.480000e-11,172.200,8.292683e+07,9.447804e+07


In [6]:
g = df.Genre.str.get_dummies()
d = df.distributor.str.get_dummies()
direc = df.director.str.get_dummies()
t = df.title.str.get_dummies()
la = df.leadActor.str.get_dummies()

In [7]:
final_data = pd.concat([df,t,d,g,direc,la], axis = 1)

In [8]:
final_data

,Id,title,distributor,Genre,leadActor,Actor popularity,director,Director popularity,Sentiment,num_theaters,...,Zachary Gordon,Zero Mostel,Zhang Ziyi,Zoe Saldana,see Cast,stand-up comedy of Steve Harvey,the American spirit,the Mid-Ocean Ridge,voices of Craig T. Nelson,voices of Don Johnson
0,3468,Whipped,Destination,Comedy,Amanda Peet,7.599365,Peter Cohen,1.120000,-1,1581,...,0,0,0,0,0,0,0,0,0,0
1,3384,Little Nicky,New Line,Comedy,Adam Sandler,7.883535,Steven Brill,1.841047,-1,2910,...,0,0,0,0,0,0,0,0,0,0
2,3381,The Ladies Man,Paramount,Comedy,Tim Meadows,1.939280,Reginald Hudlin,1.618813,-1,2022,...,0,0,0,0,0,0,0,0,0,0
3,3448,The Tao of Steve,Sony Classics,Comedy,Donal Logue,3.748402,Jenniphr Goodman,1.400000,1,9,...,0,0,0,0,0,0,0,0,0,0
4,3455,Turn It Up,New Line,Music Drama,Pras,1.156000,Robert Adetuyi,1.178253,1,661,...,0,0,0,0,0,0,0,0,0,0
5,3337,Cecil B. Demented,Artisan,Comedy,Melanie Griffith,4.050260,John Waters,2.655440,1,9,...,0,0,0,0,0,0,0,0,0,0
6,3344,The Crew,Buena Vista,Crime Comedy,Richard Dreyfuss,2.677116,Michael Dinner,1.063944,1,1510,...,0,0,0,0,0,0,0,0,0,0
7,3347,Down to You,Miramax,Romantic Comedy,Freddie Prinze,2.818751,Kris Isacsson,1.008891,1,1971,...,0,0,0,0,0,0,0,0,0,0
8,3465,Whatever It Takes,Sony / Columbia,Romantic Comedy,Shane West,5.635719,David Raynr,1.004854,1,2272,...,0,0,0,0,0,0,0,0,0,0
9,3369,Hanging Up,Sony / Columbia,Comedy / Drama,Meg Ryan,4.863958,Diane Keaton,6.070353,1,2618,...,0,0,0,0,0,0,0,0,0,0


In [9]:
final_data.drop(['Id','title','distributor','Genre','leadActor','director'],axis = 1, inplace=True)             

In [10]:
final_data

,Actor popularity,Director popularity,Sentiment,num_theaters,runtime_mins,releaseYear,CULT_INDEX,CPI,prod_budget_ADJ,rev_totalGross_ADJ,...,Zachary Gordon,Zero Mostel,Zhang Ziyi,Zoe Saldana,see Cast,stand-up comedy of Steve Harvey,the American spirit,the Mid-Ocean Ridge,voices of Craig T. Nelson,voices of Don Johnson
0,7.599365,1.120000,-1,1581,82.0,2000,5.530000e-10,172.200,4.146341e+06,1.090221e+07,...,0,0,0,0,0,0,0,0,0,0
1,7.883535,1.841047,-1,2910,84.0,2000,5.260000e-11,172.200,1.174797e+08,1.034267e+08,...,0,0,0,0,0,0,0,0,0,0
2,1.939280,1.618813,-1,2022,84.0,2000,2.290000e-10,172.200,3.317073e+07,3.568550e+07,...,0,0,0,0,0,0,0,0,0,0
3,3.748402,1.400000,1,9,87.0,2000,5.730000e-05,172.200,1.415128e+06,1.137976e+07,...,0,0,0,0,0,0,0,0,0,0
4,1.156000,1.178253,1,661,87.0,2000,5.670000e-09,172.200,1.796748e+07,3.270542e+06,...,0,0,0,0,0,0,0,0,0,0
5,4.050260,2.655440,1,9,88.0,2000,8.830000e-06,172.200,4.186671e+05,3.366715e+06,...,0,0,0,0,0,0,0,0,0,0
6,2.677116,1.063944,1,1510,88.0,2000,5.250000e-10,172.200,5.252033e+07,3.413001e+07,...,0,0,0,0,0,0,0,0,0,0
7,2.818751,1.008891,1,1971,89.0,2000,1.760000e-10,172.200,4.837398e+07,5.259552e+07,...,0,0,0,0,0,0,0,0,0,0
8,5.635719,1.004854,1,2272,94.0,2000,2.290000e-10,172.200,4.422764e+07,2.292010e+07,...,0,0,0,0,0,0,0,0,0,0
9,4.863958,6.070353,1,2618,95.0,2000,7.480000e-11,172.200,8.292683e+07,9.447804e+07,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X = final_data.tail(1)

In [12]:
m.predict(X)

KeyError: 1